In [1]:
import torch
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
# def f(x):
#     return x**2 + np.sin(x) + np.exp(x)

In [4]:
# def integral(f, a, b, N):
#     deltax = (b-a)/N
#     f_v = np.vectorize(f)
#     f_evals = f_v(np.linspace(a, b, N+1, endpoint=True))
# #     f_evals = np.array([f_v(i) for i in np.linspace(a, b, N+1, endpoint=True)])
#     value = deltax*(f_evals[0] + 2*f_evals[1:-1].sum() + f_evals[-1])/2
#     return value
# # print((b**3 - a**3)/3)matplotlib

In [5]:
# import matplotlib.pyplot as plt

In [6]:
# n_vals = range(5, 100, 5)
# evals = [integral(f, 0, 10.5, n) for n in n_vals]
# plt.plot(n_vals, evals)

In [7]:
# print(evals)

In [8]:
cd '/home/abhishekshar/hierarchichal_point_process/src/'

/home/abhishekshar/hierarchichal_point_process/src


In [9]:
from hrmtpp_exact import hrmtpp_exact

In [10]:
model = hrmtpp_exact(marker_dim=217).to(device)

In [18]:
from utils.data_loader import load_data

mimic_x, mimic_t = load_data('../data/mimic_train.pkl')

In [19]:
t = torch.from_numpy(np.stack([mimic_t[2], mimic_t[3]], axis=1)).float().to(device)[:,:,:2]
x = torch.from_numpy(np.stack([mimic_x[2], mimic_x[3]], axis=1)).float().to(device)

In [38]:
def compute_point_log_likelihood(h, d_js):
        """
            Input:
                h : Tensor of shape TxBSxCx self.shared_output_layers[-1]
                tj : Tensor of shape TxBSx2 [j,:,0] represents actual time at timestep j ,\
                    [i,:,1] represents time gap d_i = t_i- t_{i-1}
                d_js: Tensor of shape TxBSx1x(N+1) [j,:,:,k] represents the k*delta_x in trapezoidal rule
                [j,:,:,0] is 0
                https://en.wikipedia.org/wiki/Trapezoidal_rule#Uniform_grid
            Output:
                log_f_t : tensor of shape TxBSxC

        """
        
        
#         d_js = tj[:, :, 0:1, None]  # Shape TxBSx1xN Time differences

        past_influence = model.h_influence(h)  # TxBSxCx1

        # TxBSx1x1
        current_influence = model.time_influence * d_js
        base_intensity = model.base_intensity  # 1x1x1x1

        term1 = past_influence + current_influence + base_intensity
        term2 = (past_influence + base_intensity).exp()
        term3 = term1.exp()

        log_f_t = term1 + \
            (1./(model.time_influence+1e-6 )) * (term2-term3)
        return log_f_t  # TxBSxCx(N+1)

    
with torch.no_grad():
    a = t[:, :, 0:1, None]
    N = 100
    dmax = t.max() - t.min()
    delta_x = (a + dmax/10 - a)/N
    d_js = torch.arange(0, N+1).float().to(device)*delta_x
    times = a + d_js
    hs = model.run_forward_backward_rnn(x, t, None)
    hz_embedded = model.preprocess_hidden_latent_state(hs)
    time_log_likelihood = compute_point_log_likelihood(hz_embedded, d_js) # TxBSxC

    
    # t.f(t)
    g = (times.log() + time_log_likelihood).exp()
    integral = g[:,:,:,:1] + 2 * torch.sum(g[:,:,:,1:-1], dim=-1, keepdim=True) + g[:,:,:,-1:]
    integral = 0.5 * delta_x * integral

In [39]:
times.shape, 

(torch.Size([3, 2, 1, 101]),)

In [40]:
integral.shape

torch.Size([3, 2, 5, 1])

In [41]:
time_log_likelihood

tensor([[[[ 7.4486e-03, -1.0391e+02, -3.1650e+02,  ..., -2.0832e+32,
           -4.2468e+32, -8.6578e+32],
          [-3.6657e-02, -9.9441e+01, -3.0283e+02,  ..., -1.9933e+32,
           -4.0636e+32, -8.2842e+32],
          [-2.2586e-01, -8.2372e+01, -2.5058e+02,  ..., -1.6497e+32,
           -3.3631e+32, -6.8562e+32],
          [ 5.7212e-02, -1.0920e+02, -3.3268e+02,  ..., -2.1895e+32,
           -4.4635e+32, -9.0996e+32],
          [-5.7042e-02, -9.7441e+01, -2.9671e+02,  ..., -1.9531e+32,
           -3.9816e+32, -8.1171e+32]],

         [[ 7.4486e-03, -1.0391e+02, -3.1650e+02,  ..., -2.0830e+32,
           -4.2465e+32, -8.6571e+32],
          [-3.6657e-02, -9.9441e+01, -3.0283e+02,  ..., -1.9931e+32,
           -4.0633e+32, -8.2836e+32],
          [-2.2586e-01, -8.2372e+01, -2.5058e+02,  ..., -1.6496e+32,
           -3.3629e+32, -6.8557e+32],
          [ 5.7212e-02, -1.0920e+02, -3.3268e+02,  ..., -2.1893e+32,
           -4.4632e+32, -9.0989e+32],
          [-5.7042e-02, -9.7441e+01

In [42]:
delta_x

tensor([[[[71.2287]],

         [[71.2286]]],


        [[[71.2287]],

         [[71.2287]]],


        [[[71.2287]],

         [[71.2287]]]], device='cuda:0')

In [43]:
dmax

tensor(71228.6328, device='cuda:0')